<a href="https://colab.research.google.com/github/godpeny/laboratory/blob/master/Study/Deep_Learning_for_Everyone/Chapter_4/sonar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install
%pip install pandas
%pip install tensorflow
%pip install sklearn

In [24]:
# import
import pandas as pd

from google.colab import drive

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split, KFold

In [3]:
# using Google Drive
drive.mount('/d')
file_path = '/d/My Drive/Colab Notebooks/DL-tutorial/data/sonar3.csv'

df = pd.read_csv(file_path, header=None)
df.head(3)

Drive already mounted at /d; to attempt to forcibly remount, call drive.mount("/d", force_remount=True).


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0


In [4]:
df[len(df.columns) -1].value_counts()

x = df.iloc[:, 0:60]
y = df.iloc[:, 60]

In [5]:
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# history = model.fit(x, y, epochs=100, batch_size=5) # the result is overfitted

In [7]:
# split train and test sets in order to avoid overfitting
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

In [8]:
history_fit = model.fit(x_train, y_train, epochs=100, batch_size=5) # using only test sets
score = model.evaluate(x_test, y_test) # evaluate with test sets
print(score)

Epoch 1/100
29/29 [==============================] - 1s 2ms/step - loss: 0.6434 - accuracy: 0.6966
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 0.6102 - accuracy: 0.7448
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5940 - accuracy: 0.7724
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5722 - accuracy: 0.7931
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5567 - accuracy: 0.7862
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5395 - accuracy: 0.7586
Epoch 7/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.7931
Epoch 8/100
29/29 [==============================] - 0s 2ms/step - loss: 0.5167 - accuracy: 0.7793
Epoch 9/100
29/29 [==============================] - 0s 2ms/step - loss: 0.4977 - accuracy: 0.8069
Epoch 10/100
29/29 [==============================] - 0s 2ms/step - loss: 0.4838 - accuracy: 0.8000
Epoch 11/

In [10]:
# save model in Google Drive
model_path='/d/My Drive/Colab Notebooks/DL-tutorial/models/sonar.hdf5'
model.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# delete model from memory and reload from Google Drive
del model
model = load_model(model_path)

In [14]:
# test with loaded model
score = model.evaluate(x_test, y_test)
print(score)

2/2 [==============================] - 0s 9ms/step - loss: 0.5229 - accuracy: 0.7937
[0.5229355692863464, 0.7936508059501648]


In [29]:
# using KFold to best use of train and data sets
k = 5
kfold = KFold(n_splits=k, shuffle=True)
acc_score = []

def model_fn(): # make new model in order to begin with train.
  model = Sequential()
  model.add(Dense(24, input_dim=60, activation='relu'))
  model.add(Dense(6, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [30]:
for train_i, test_i in kfold.split(x):
  x_train, x_test = x.iloc[train_i, :], x.iloc[test_i, :]
  y_train, y_test = y.iloc[train_i], y.iloc[test_i]

  model = model_fn()
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
  model.fit(x, y, epochs=100, batch_size=5)

  acc = model.evaluate(x_test, y_test)[1] # [0] = loss, [1] = accuracy
  acc_score.append(acc) # append accuracy to list

Epoch 1/100
42/42 [==============================] - 1s 2ms/step - loss: 0.6880 - accuracy: 0.5529
Epoch 2/100
42/42 [==============================] - 0s 2ms/step - loss: 0.6708 - accuracy: 0.6058
Epoch 3/100
42/42 [==============================] - 0s 2ms/step - loss: 0.6600 - accuracy: 0.6490
Epoch 4/100
42/42 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.6731
Epoch 5/100
42/42 [==============================] - 0s 2ms/step - loss: 0.6349 - accuracy: 0.6779
Epoch 6/100
42/42 [==============================] - 0s 3ms/step - loss: 0.6209 - accuracy: 0.6683
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.6075 - accuracy: 0.7163
Epoch 8/100
42/42 [==============================] - 0s 3ms/step - loss: 0.5951 - accuracy: 0.7163
Epoch 9/100
42/42 [==============================] - 0s 2ms/step - loss: 0.5771 - accuracy: 0.7452
Epoch 10/100
42/42 [==============================] - 0s 2ms/step - loss: 0.5652 - accuracy: 0.7356
Epoch 11/

In [31]:
print(acc_score) # total accuracy score
print(sum(acc_score) /k ) # avg accuracy score

[1.0, 0.9523809552192688, 0.976190447807312, 0.9756097793579102, 0.9756097793579102]
0.9759581923484802
